# On this Jupyter Notebook 
Creation date: February 12, 2024

For satisfactory Bus Delay Data store see "./data/delays/bus-delay-store.csv".

This Jupyter Notebook is intended to perform basic data analysis on TTC delay data, with the goal of combining data from multiple excel spreadsheets and, within them, multiple sheets such that there is one usable CSV file containing a specific transportation system's delay data, ordered by their recorded Date and Time stamp. 

This Jupyter Notebook must be in the home directory to be run.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import json
from tqdm.notebook import tqdm

In [4]:
### Update as of 2025-02-22 from Cindy: reorganized the direcotry and folders to distinguish bus and subway data. 
### Need to be careful with the file path.
suwbay_prefix = "./data/delays/subway/ttc-subway-delay-data-"
file_names = []
for i in range(2016, 2025):
    file_names.append(f"{subway_prefix}{i}.xlsx")

- Depending on which dataset is used, the function calls would be different. 
- The distinction is made because the column_data is different for bus and subway. 

### Bus 

In [4]:
def combine_all_sheets_to_csv(file_names, output_csv="combined.csv"):
    """
    Reads all sheets from every Excel file in file_names (without interpreting any row as header),
    concatenates the data from all sheets vertically, and exports the combined data to a CSV file
    without any column headers.

    Parameters:
        file_names (list of str): List of paths to Excel files.
        output_csv (str): The path for the output CSV file.
    """
    all_data = []

    column_data = ['Report Date','Route','Time','Day','Location','Incident', 'Min Delay','Min Gap','Direction','Vehicle']

    for file in file_names:
        # Open the excel sheet
        try:
            excel_file = pd.ExcelFile(file)
        except Exception as e:
            print(f"Error opening file '{file}': {e}")
            continue

        # Read and append values for each sheet in the file
        for sheet in excel_file.sheet_names:
            try:
                # Header is first row
                dummy = pd.read_excel(excel_file, sheet_name=sheet, header=0)
                dummy.columns = column_data
                all_data.append(dummy)
            except Exception as e:
                print(f"Error reading sheet '{sheet}' in file '{file}': {e}")
                continue

    if not all_data:
        print("There was no data in any file given or the passed file_name vector was empty")
        return

    # Concat all the data vertically
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df.columns = column_data

    try:
        combined_df.to_csv(output_csv, header=column_data, index=False)
        print(f"Combined data successfully exported to '{output_csv}'.")
    except Exception as e:
        print(f"Error exporting data to CSV: {e}")

In [9]:
# Combine all sheets to csv file and process the csv file
combine_all_sheets_to_csv(file_names, output_csv="./data/delays/bus-delay-data-2016-2024.csv")
# !!!!!!!!!!!!!!!!!!DATE TIME PROCESSING MUST BE DONE LOCALLY IN EXCEL THIS NEARLY CRASHED MY COMPUTER!!!!!!!!!!!!!!!!!

Combined data successfully exported to './data/delays/bus-delay-data-2016-2024.csv'.


### Subway

In [11]:
import pandas as pd
import os

def combine_subway_sheets_to_csv(file_names, output_csv="./data/delays/subway-delay-data-2016-2024.csv"):
    """
    Reads all sheets from every subway delay Excel file, concatenates them, and exports the combined data to a CSV file.

    Parameters:
        file_names (list of str): List of relative paths to Excel files.
        output_csv (str): Relative path for the output CSV file.
    """
    all_data = []

    # Updated column names for the subway dataset
    column_data = ['Date', 'Time', 'Day', 'Station', 'Code', 'Min Delay', 'Min Gap', 'Bound', 'Line', 'Vehicle']

    for file in file_names:
        try:
            excel_file = pd.ExcelFile(file)
        except Exception as e:
            print(f"Error opening file '{file}': {e}")
            continue

        for sheet in excel_file.sheet_names:
            try:
                dummy = pd.read_excel(excel_file, sheet_name=sheet, header=0)
                dummy.columns = column_data
                all_data.append(dummy)

            except Exception as e:
                print(f"Error reading sheet '{sheet}' in file '{file}': {e}")
                continue

    if not all_data:
        print("No data found in given files.")
        return

    # Concatenate all sheets together
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df.columns = column_data

    try:
        combined_df.to_csv(output_csv, header=True, index=False)
        print(f"Combined data successfully exported to '{output_csv}'.")
    except Exception as e:
        print(f"Error exporting data to CSV: {e}")

subway_prefix = "/Users/jessica_1/Workspace/EngSci_Year3/ECE324/TardyTerminatorCommittee/data/delays/bus/subway/ttc-subway-delay-data-"
file_names = [f"{subway_prefix}{i}.xlsx" for i in range(2016, 2025)]

# Run the function 
combine_subway_sheets_to_csv(file_names, output_csv="/Users/jessica_1/Workspace/EngSci_Year3/ECE324/TardyTerminatorCommittee/data/delays/bus/subway/ttc-subway-delay-data-2014-2024.csv")

Combined data successfully exported to '/Users/jessica_1/Workspace/EngSci_Year3/ECE324/TardyTerminatorCommittee/data/delays/bus/subway/ttc-subway-delay-data-2014-2024.csv'.


# Please complete DateTime Processing MANUALLY in EXCEL before proceeding

In [13]:
def pre_processing_full_ttc_csv(file_name, dict_file_name = "info.json"): 
    '''
    Function for processing TTC delay data stored in a CSV file. This function must be run prior to further analysis to allow for some
    functions to run smoothly.
    Manual grunt work may be needed for certain data cleaning, which is why print statements are added for some brief manual parsing

    Input: file name of a TTC Delay CSV File; (optional) file name of a data_dict dump file, if empty it is defaulted to 'info.json'
    Output: returns a dictionary of data for the specified file_name (unique categoricies for a feature, etc)

    Note: the Data Dictionary is also stored as a JSON file for later use if needed
    '''
    df = pd.read_csv("/Users/jessica_1/Workspace/EngSci_Year3/ECE324/TardyTerminatorCommittee/data/delays/bus/subway/ttc-subway-delay-data-2014-2024 - ttc-subway-delay-data-2014-2024.csv")
    column_data = df.columns
    data_dict = {}
    for col in column_data:
        df[col] = df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)
        if isinstance(df[col][1], str) and col != 'DateTime' and col != 'Report Date' and col != 'Time' and col != 'Day':
            data_dict[col] = df[col].unique().tolist()
    
    # df.to_csv(file_name, index='DateTime')
    
    # with open(dict_file_name, "w") as outfile: 
    #     json.dump(data_dict, outfile)
        
    return data_dict

In [7]:
# for Bus Data
data_dict = pre_processing_full_ttc_csv("./data/delays/bus-delay-final.csv", dict_file_name= "./data/delays/bus-delay-data.json")

/var/folders/y2/255cmf8x457_1wfw5bhh8cwm0000gp/T/ipykernel_18494/135831970.py:12: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


In [14]:
# for subway data
data_dict = pre_processing_full_ttc_csv("./data/delays/subway-delay-data-2016-2024.csv", dict_file_name="./data/delays/subway-delay-data.json")


## On Bus delay data
Observations of unique bus delay locations from preprocessed data (see "./data/delays/bus-delay-data.json") indicates an exorbant number of typos present, especially when compared to route station names in "data/TTC Routes and Schedules Data/stops.txt" (retrieved from https://open.toronto.ca/dataset/ttc-routes-and-schedules/). These files are pushed to github in a zip folder as they are very large. They are in the gitignore.

Further processing (textual) is necessary for ease of analysis in future steps. As texts must be done in a 1-1 manner, a LLM processing method would be used, aligned with that as described in Lecture #10 with Gemini 2.0 Flash. 

Please see "./preliminary/stop_name_processing.py" for further details of this process.

